In [18]:
#you'll need to pip install a few things (again this is in linux but it shouldn't be too hard to install these packages in mac)
# !pip install mido
# !pip install midi2audio
# !sudo apt-get install fluidsynth

In [19]:
import mido
from mido import MidiFile, MidiTrack, Message, MetaMessage, bpm2tempo, tempo2bpm, second2tick, tick2second
from midi2audio import FluidSynth
from IPython.display import Audio
import os

In [20]:
NOTE_NAMES = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]

def get_note_name(midi_note):
    """Get the note name from a MIDI note number."""
    return NOTE_NAMES[midi_note % 12] + str(midi_note // 12 - 1)  # Corrected the octave offset


In [21]:
get_note_name(60)

'C4'

In [ ]:
def generate_chords(start, end, chord_type='', num_notes=3):
    if chord_type not in ['major', 'minor', 'dom7', 'diminished', 'augmented', 'sus2', 'sus4']:
        raise ValueError('Invalid chord type. Must be one of "major", "minor", "dom7", "diminished", "augmented", "sus2", "sus4".')

    chords = []
    for root in range(start, end+1):
        if chord_type == 'major':
            chord = [root, root + 4, root + 7]  # Major triad
            if num_notes >= 4:
                chord.append(root + 11)  # Major seventh
            if num_notes >= 5:
                chord.append(root + 14)  # Major ninth
        elif chord_type == 'minor':
            chord = [root, root + 3, root + 7]  # Minor triad
            if num_notes >= 4:
                chord.append(root + 10)  # Minor seventh
            if num_notes >= 5:
                chord.append(root + 14)  # Minor ninth
        elif chord_type == 'dom7':
            chord = [root, root + 4, root + 7, root + 10]  # Dominant 7th
            if num_notes >= 5:
                chord.append(root + 14)  # Dominant 9th
        elif chord_type == 'diminished':
            chord = [root, root + 3, root + 6]
        elif chord_type == 'augmented':
            chord = [root, root + 4, root + 8]
        elif chord_type == 'sus2':
            chord = [root, root + 2, root + 7]
        elif chord_type == 'sus4':
            chord = [root, root + 5, root + 7]
        chords.append(chord)
    return chords

In [ ]:
#a total of 13 types of chords (7 basic chords + 6 extended chords)

start_note = 21  # A0
end_note = 108  # C8

# Generate all major chords (triads, sevenths, ninths)
major_triads = generate_chords(start_note, end_note, chord_type='major',num_notes=3)
major_sevenths = generate_chords(start_note, end_note, chord_type='major',num_notes=4)
major_ninths = generate_chords(start_note, end_note, chord_type='major',num_notes=5)

#generate all minor chords (triads, sevenths, ninths)
minor_triads = generate_chords(start_note, end_note, chord_type='minor',num_notes=3)
minor_sevenths = generate_chords(start_note, end_note, chord_type='minor',num_notes=4)
minor_ninths = generate_chords(start_note, end_note, chord_type='minor',num_notes=5)

#gene rate all dominant 7th chords (sevenths, ninths)
dom7_sevenths = generate_chords(start_note, end_note, chord_type='dom7',num_notes=4)
dom7_ninths = generate_chords(start_note, end_note, chord_type='dom7',num_notes=5)

#generate all diminished chords (triads)
dim_triads = generate_chords(start_note, end_note, chord_type='diminished',num_notes=3)

#generate all augmented chords (triads)
aug_triads = generate_chords(start_note, end_note, chord_type='augmented',num_notes=3)

#generate all sus2 chords (triads)
sus2_triads = generate_chords(start_note, end_note, chord_type='sus2',num_notes=3)

#generate all sus4 chords (triads)
sus4_triads = generate_chords(start_note, end_note, chord_type='sus4',num_notes=3)

In [60]:
fs = FluidSynth('/home/dofo/Desktop/music/soundfonts/Yamaha_MODX8_piano_fix/Yamaha MODX8 piano.sf2') #change this to the path of the sf2 file

def play_chord(chord, chord_name, directory=''):
    # Ensure the directory exists
    os.makedirs(directory, exist_ok=True)

    for i, chord in enumerate(chord):
        # Create a new MIDI file
        mid = MidiFile()
        track = MidiTrack()
        mid.tracks.append(track)

        # Change to a different instrument (0 = Acoustic Grand Piano)
        track.append(Message('program_change', program=0, time=0))

        # Add the notes of the chord
        for note in chord:
            track.append(Message('note_on', note=note, velocity=64, time=0))
        track.append(Message('note_off', note=note, velocity=64, time=1920))
        for note in chord:
            track.append(Message('note_off', note=note, velocity=64, time=0))

        # Extract the root note name and its MIDI number
        root_note_name = get_note_name(chord[0])
        root_midi_number = chord[0]

        # Save the MIDI file with a descriptive name
        midi_filename = os.path.join(directory, f'{chord_name}_{root_note_name}_{root_midi_number}.mid')
        mid.save(midi_filename)

        # Convert the MIDI file to a WAV file
        wav_filename = os.path.join(directory, f'{chord_name}_{root_note_name}_{root_midi_number}.wav')
        fs.midi_to_audio(midi_filename, wav_filename)

        # Clean up the MIDI file
        os.remove(midi_filename)



In [30]:
def test_chord(start, chord_type, num_notes):
    if chord_type not in ['test']:
        raise ValueError('chord_type must be one of: test')
    
    chords = []
    root = start
    test_chord = [root, root+4, root+7]
    chords.append(test_chord)
    return chords


In [32]:
test1 = test_chord(60, 'test', 3)
test1[0]

[60, 64, 67]

In [27]:
fs = FluidSynth('/home/dofo/Desktop/music/soundfonts/NordGrand1/NordGrand1.sf2') #change this to the path of the sf2 file


def play_test(chord, chord_name, directory='/home/dofo/Desktop/music/test_chords'):
    # Ensure the directory exists
    os.makedirs(directory, exist_ok=True)
    
    # Create a new MIDI file
    mid = MidiFile()
    track = MidiTrack()
    mid.tracks.append(track)

    # Change to a different instrument (0 = Acoustic Grand Piano)
    track.append(Message('program_change', program=0, time=0))

    # Add the notes of the chord
    for note in chord[0]:  # You need to index into the list of lists returned by `test_chord`
        track.append(Message('note_on', note=note, velocity=64, time=0))

    # Delay between note on and note off
    track.append(Message('note_off', note=note, velocity=64, time=1920))

    for note in chord[0]:  # You need to index into the list of lists returned by `test_chord`
        track.append(Message('note_off', note=note, velocity=64, time=0))

    # Save the MIDI file
    midi_filename = os.path.join(directory, f'{chord_name}.mid')
    mid.save(midi_filename)

    # Convert the MIDI file to a WAV file
    wav_filename = os.path.join(directory, f'{chord_name}.wav')
    fs.midi_to_audio(midi_filename, wav_filename)

    # Clean up the MIDI file
    os.remove(midi_filename)

In [28]:
play_test(test_chords, 'Nord_test_chord', directory='/home/dofo/Desktop/music/test_chords')

NameError: name 'test_chords' is not defined

In [61]:
#hello?
play_chord(major_triads, 'MODX8_major_triad', directory='/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/major_triad')
play_chord(major_sevenths, 'MODX8_major_seventh', directory='/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/major_seventh')
play_chord(major_ninths, 'MODX8_major_ninth', directory='/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/major_ninth')


fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]


FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/major_triad/MODX8_major_triad_A0_21.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/major_triad/MODX8_major_triad_A#0_22.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/major_triad/MODX8_major_triad_B0_23.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-20

fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]


FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/major_triad/MODX8_major_triad_D#1_27.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/major_triad/MODX8_major_triad_E1_28.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/major_triad/MODX8_major_triad_F1_29.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-20

fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]


...

FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/major_ninth/MODX8_major_ninth_G6_91.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/major_ninth/MODX8_major_ninth_G#6_92.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/major_ninth/MODX8_major_ninth_A6_93.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-20

fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]


FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/major_ninth/MODX8_major_ninth_C#7_97.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/major_ninth/MODX8_major_ninth_D7_98.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/major_ninth/MODX8_major_ninth_D#7_99.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2

fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]


FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/major_ninth/MODX8_major_ninth_G7_103.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/major_ninth/MODX8_major_ninth_G#7_104.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/major_ninth/MODX8_major_ninth_A7_105.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000

In [62]:
#Minor chords
play_chord(minor_triads, 'MODX8_minor_triad', directory='/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/minor_triad')
play_chord(minor_sevenths, 'MODX8_minor_seventh', directory='/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/minor_seventh')
play_chord(minor_ninths, 'MODX8_minor_ninth', directory='/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/minor_ninth')

FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/minor_triad/MODX8_minor_triad_A0_21.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/minor_triad/MODX8_minor_triad_A#0_22.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/minor_triad/MODX8_minor_triad_B0_23.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-20

fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]


FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/minor_triad/MODX8_minor_triad_C#1_25.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/minor_triad/MODX8_minor_triad_D1_26.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/minor_triad/MODX8_minor_triad_D#1_27.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2

fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]


FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/minor_triad/MODX8_minor_triad_G1_31.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/minor_triad/MODX8_minor_triad_G#1_32.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/minor_triad/MODX8_minor_triad_A1_33.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-20

...

fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]


FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/minor_ninth/MODX8_minor_ninth_G#6_92.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/minor_ninth/MODX8_minor_ninth_A6_93.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/minor_ninth/MODX8_minor_ninth_A#6_94.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2

fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]


FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/minor_ninth/MODX8_minor_ninth_D7_98.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/minor_ninth/MODX8_minor_ninth_D#7_99.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/minor_ninth/MODX8_minor_ninth_E7_100.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2

fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]


In [63]:
#Dominant 7th chords
play_chord(dom7_sevenths, 'MODX8_dom7_seventh', directory='/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/dom7_seventh')
play_chord(dom7_ninths, 'MODX8_dom7_ninth', directory='/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/dom7_ninth')

fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]


FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/dom7_seventh/MODX8_dom7_seventh_A0_21.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/dom7_seventh/MODX8_dom7_seventh_A#0_22.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/dom7_seventh/MODX8_dom7_seventh_B0_23.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2

fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]


FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/dom7_seventh/MODX8_dom7_seventh_D#1_27.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/dom7_seventh/MODX8_dom7_seventh_E1_28.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/dom7_seventh/MODX8_dom7_seventh_F1_29.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2

fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]


...

fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]


FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/dom7_ninth/MODX8_dom7_ninth_B6_95.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/dom7_ninth/MODX8_dom7_ninth_C7_96.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/dom7_ninth/MODX8_dom7_ninth_C#7_97.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Pet

fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]


FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/dom7_ninth/MODX8_dom7_ninth_E7_100.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/dom7_ninth/MODX8_dom7_ninth_F7_101.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/dom7_ninth/MODX8_dom7_ninth_F#7_102.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 

fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]


In [64]:
#Diminished chords
play_chord(dim_triads, 'MODX8_diminished', directory='/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/diminished')

#Augmented chords
play_chord(aug_triads, 'MODX8_augmented', directory='/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/augmented')

#Sus2 chords
play_chord(sus2_triads, 'MODX8_sus2', directory='/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/sus2')

#Sus4 chords
play_chord(sus4_triads, 'MODX8_sus4', directory='/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/sus4')

FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/diminished/MODX8_diminished_A0_21.wav'..


fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]


FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/diminished/MODX8_diminished_A#0_22.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/diminished/MODX8_diminished_B0_23.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/diminished/MODX8_diminished_C1_24.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Pet

fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]


FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/diminished/MODX8_diminished_E1_28.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/diminished/MODX8_diminished_F1_29.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/diminished/MODX8_diminished_F#1_30.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Pet

...

fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]


FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/sus4/MODX8_sus4_B6_95.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/sus4/MODX8_sus4_C7_96.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/sus4/MODX8_sus4_C#7_97.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed u

fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]


FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/sus4/MODX8_sus4_F7_101.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/sus4/MODX8_sus4_F#7_102.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords/sus4/MODX8_sus4_G7_103.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distribute

fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
